In [9]:
import pandas as pd


csvPath =  "./Resources/purchase_data.csv"
itemPD_df = pd.read_csv(csvPath)

#Player count~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#counts only unique by "Screen Name" rows
playerCount = itemPD_df["SN"].nunique()

#Display
print(f"Total Players: {playerCount}")

#Purchasing  Analysis (total)~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#create series to store Purchasing Analysis data
pAnalysis = pd.Series({
    "Number of Unique Items":0,
    "Average Purchase Price": float(0),
    "Total Number of Purchases":0,
    "Total Revenue":0
})

#finds the number of unique items in the data set and stores it in series
pAnalysis["Number of Unique Items"] = itemPD_df["Item Name"].nunique()

#finds the mean of the price column from the data set and stores it in series
pAnalysis["Average Purchase Price"] = itemPD_df["Price"].mean()

#counts the total number of rows within the data set and stores that in series
pAnalysis["Total Number of Purchases"] = itemPD_df["Price"].count()

#adds price column together and stores it in series
pAnalysis["Total Revenue"] = itemPD_df["Price"].sum()

#Display
print(pAnalysis)

#Gender Demographics~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#create series to store Gender Demographics
gDemo = pd.Series({
    "Male Players":0,
    "Male Player Percentage":0,
    "Female Players":0,
    "Female Player Percentage":0,
    "Other/Non-Disclosed Players":0,
    "Other/Non-Disclosed Player Percentage":0,
})

#remove duplicate players from df based on Screen Name
gDemoDupless_df = itemPD_df.drop_duplicates(subset="SN")

#sums the number of "Male" occurences in dupeless df then adds to series
gDemo["Male Players"] = (gDemoDupless_df.Gender == "Male").sum()

#Calculates percentage of players that are male
gDemo["Male Player Percentage"] = (gDemo["Male Players"] / playerCount) * 100

#sums the number of "Female" occurences in dupeless df then adds to series
gDemo["Female Players"] = (gDemoDupless_df.Gender == "Female").sum()

#Calculates percentage of players that are female
gDemo["Female Player Percentage"] = (gDemo["Female Players"] / playerCount) * 100

#sums the number of "Other/Non-Disclosed" occurences in dupeless df then adds to series
gDemo["Other/Non-Disclosed Players"] = (gDemoDupless_df.Gender == "Other / Non-Disclosed").sum()

#Calculates percentage of players that are Other/Non-Disclosed
gDemo["Other/Non-Disclosed Percentage"] = (gDemo["Other/Non-Disclosed Players"] / playerCount) * 100

#Display
print(gDemo)

#Purchasing Analysis(Gender)~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#create DF to store purchasing analysis by gender
gPA_df = pd.DataFrame({"Gender":["Female", "Male","Other/Non-Disclosed"],
                      "Purchase Count":[0,0,0],
                      "Average Purchase Price":[0,0,0],
                      "Total Purchase Value":[0,0,0],
                      "Avg Total Purchase per Person":[0,0,0]    
                     })

#set index to be gender
gPA_df.set_index("Gender", inplace=True)

#Sets values data frame values for males
gPA_df.loc["Male","Purchase Count"] = (itemPD_df.Gender == "Male").sum()
gPA_df.loc["Male","Average Purchase Price"] = itemPD_df.query('Gender == "Male"').Price.sum() / gPA_df.loc["Male","Purchase Count"]
gPA_df.loc["Male","Total Purchase Value"] = itemPD_df.query('Gender == "Male"').Price.sum()
gPA_df.loc["Male","Avg Total Purchase per Person"] = gPA_df.loc["Male","Total Purchase Value"] / gDemo["Male Players"]

#sets data frame values for females
gPA_df.loc["Female","Purchase Count"] = (itemPD_df.Gender == "Female").sum()
gPA_df.loc["Female","Average Purchase Price"] = itemPD_df.query('Gender == "Female"').Price.sum() / gPA_df.loc["Female","Purchase Count"]
gPA_df.loc["Female","Total Purchase Value"] = itemPD_df.query('Gender == "Female"').Price.sum()
gPA_df.loc["Female","Avg Total Purchase per Person"] = gPA_df.loc["Female","Total Purchase Value"] / gDemo["Female Players"]

#sets data frame values for other/non-disclosed
gPA_df.loc["Other/Non-Disclosed","Purchase Count"] = (itemPD_df.Gender == "Other / Non-Disclosed").sum()
gPA_df.loc["Other/Non-Disclosed","Average Purchase Price"] = itemPD_df.query('Gender == "Other / Non-Disclosed"').Price.sum() / gPA_df.loc["Other/Non-Disclosed","Purchase Count"]
gPA_df.loc["Other/Non-Disclosed","Total Purchase Value"] = itemPD_df.query('Gender == "Other / Non-Disclosed"').Price.sum()
gPA_df.loc["Other/Non-Disclosed","Avg Total Purchase per Person"] = gPA_df.loc["Other/Non-Disclosed","Total Purchase Value"] / gDemo["Other/Non-Disclosed Players"]

#Age Demographics~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#set bin for age ranges
bins = [0,9,14,19,24,29,34,39,999]

#set labels for age ranges
group_names = ["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]

#move column df data with labels/bins to new column "Binned Age"
itemPD_df["Binned Age"] = pd.cut(itemPD_df["Age"], bins, labels=group_names)

ageGrouped_df = itemPD_df.groupby("Binned Age")

#find unique age count by comparing SN
ageCountUnique_df = ageGrouped_df["SN"].nunique()

#calc percentage of total by dividing everything by total players in game stored in var "playerCount"
agePercentageTotal_df = round(ageCountUnique_df / playerCount * 100, 2)

#move data to single df to call
ageDisplay_df = pd.DataFrame({"Total Count":ageCountUnique_df,
                              "Percentage of Players":agePercentageTotal_df
                             })
#Display
print(ageDisplay_df)

#Purchasing Analysis (Age)~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#bin purchase data frame by age
itemPD_df["Age Ranges"] = pd.cut(itemPD_df["Age"], bins, labels=group_names)

#purchase count by age bin
purchaseCount = itemPD_df["Age Ranges"].value_counts()

#avg purchase price by age bin
avgPurchasePrice = itemPD_df.groupby("Age Ranges")["Price"].mean()

#total purchase amnt by age bin
totalPurchaseValue = itemPD_df.groupby("Age Ranges")["Price"].sum()

#calc avg total purchase per person
avgTotalPurchasePer = totalPurchaseValue / purchaseCount

#creates display DF for purchase stats by age bin
purchaseAnlAge_df = pd.DataFrame({"Purchase Count": purchaseCount,
                                  "Average Purchase Price": avgPurchasePrice,
                                  "Total Purchase Value": totalPurchaseValue,
                                  "Avg Total Purchase per Person": avgTotalPurchasePer
                                 })
#Display
print(purchaseAnlAge_df)

#Top Spenders~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#Group DF by Screen Name
topPurchaseGroup = itemPD_df.groupby("SN")

#Calc each value by group
purchaseCount = topPurchaseGroup["Price"].count()
avgPurchasePrice = round(topPurchaseGroup["Price"].mean(),2)
totalPurchaseValue = round(topPurchaseGroup["Price"].sum(),2)

#creates display DF for the Top 5 spenders
Top5spenders_df = pd.DataFrame({"Purchase Count": purchaseCount,
                                "Average Purchase Price": avgPurchasePrice,
                                "Total Purchase Value": totalPurchaseValue
                               })

#sort list highest value "Total Purchase Value" on top
Top5spenders_df = Top5spenders_df.sort_values(["Total Purchase Value"], ascending = False)

#add dollar signs to appropriate columns' data
Top5spenders_df["Average Purchase Price"] = Top5spenders_df["Average Purchase Price"].map("${:,}".format)
Top5spenders_df["Total Purchase Value"] = Top5spenders_df["Total Purchase Value"].map("${:,}".format)

#Display
Top5spenders_df.head()

#Most Popular Items~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#puts item id, item name, and price columns into new df
itemData_df = itemPD_df[["Item ID", "Item Name", "Price"]]

#group df by specific item
itemDataGrouped = itemData_df.groupby(["Item ID", "Item Name"])

purchaseCount = itemDataGrouped["Price"].count()

#can't figure out how to just copy the Price data frame so doing it this way
itemPrice = itemDataGrouped["Price"].mean()

totalPurchaseValue = itemDataGrouped["Price"].sum()

#create df to display more popular item stats
mostPopItem_df = pd.DataFrame({"Purchase Count": purchaseCount,
                            "Item Price": itemPrice,
                            "Total Purchase Value": totalPurchaseValue
                              })

#sort df by "Purchase Count"
mostPopItemSort_df = mostPopItem_df.sort_values("Purchase Count", ascending = False)

#clean up decimals and add dollar signs in appropriate columns
mostPopItemSort_df["Item Price"] = mostPopItemSort_df["Item Price"].map("${:,.2f}".format)
mostPopItemSort_df["Total Purchase Value"] = mostPopItemSort_df["Total Purchase Value"].map("${:,.2f}".format)

#Display
mostPopItem_df.head()

#Most Profitable Items~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#sort prior DF by "Total Purchase Value"
mostProfItem_df = mostPopItem_df.sort_values("Total Purchase Value", ascending = False)

#clean up decimals and add dollar signs in appropriate columns
mostProfItem_df["Item Price"] = mostProfItem_df["Item Price"].map("${:,.2f}".format)
mostProfItem_df["Total Purchase Value"] = mostProfItem_df["Total Purchase Value"].map("${:,.2f}".format)

#Display
mostProfItem_df.head()


Total Players: 576
Number of Unique Items        179.000000
Average Purchase Price          3.050987
Total Number of Purchases     780.000000
Total Revenue                2379.770000
dtype: float64
Male Players                             484.000000
Male Player Percentage                    84.000000
Female Players                            81.000000
Female Player Percentage                  14.000000
Other/Non-Disclosed Players               11.000000
Other/Non-Disclosed Player Percentage      0.000000
Other/Non-Disclosed Percentage             1.909722
dtype: float64
            Total Count  Percentage of Players
Binned Age                                    
<10                  17                   2.95
10-14                22                   3.82
15-19               107                  18.58
20-24               258                  44.79
25-29                77                  13.37
30-34                52                   9.03
35-39                31                   5.38


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80
